# 🔎 Análisis Exploratorio de Datos (Google):

### Importación de librerias

In [6]:
from textblob import TextBlob
import datetime
import inspect
import re
import matplotlib.pyplot as plt
import inspect
import pandas as pd
import gzip
import os
import seaborn as sns
import numpy as np
from data_utils import data_type_check

### Importación de los datasets

In [20]:
df_metadata = pd.read_parquet('../0_Dataset/Data_Limpia/Google/G_metadata_FL.parquet', engine='pyarrow')
df_review = pd.read_parquet('../0_Dataset/Data_Limpia/Google/G_review_FL_reducido.parquet', engine='pyarrow')

### Metadata de Florida (FL)

Verificación de los datos

In [22]:
df_metadata.head(5)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1302883,Neurocare & Family Services,"Neurocare & Family Services, 1749 S Naperville...",0x880e55ddb0081907:0x1dc5f1650b07c388,None,41.839291,-88.100503,[Doctor],5.0,1,None,"[[Monday, Closed], [Tuesday, 8AM–10PM], [Wedne...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 8AM Tue,"[0x880e55ddba9f7635:0x862c597b017011c1, 0x880e...",https://www.google.com/maps/place//data=!4m2!3...
749243,Bayview Canvas,"Bayview Canvas, 22460 Glass Ln, Port Charlotte...",0x146c6dea0e8465f9:0xca5e7f14cefd4983,None,26.962494,-82.075552,[Upholstery shop],5.0,1,None,"[[Friday, 8AM–5PM], [Saturday, Closed], [Sunda...","{'Accessibility': None, 'Activities': None, 'A...",Open ⋅ Closes 5PM,"[0x88dca9b000000001:0xae92d5429c06878a, 0x88db...",https://www.google.com/maps/place//data=!4m2!3...
2190487,Peluquería Acapulco Barbershop,"Peluquería Acapulco Barbershop, 1014 Gault Ave...",0x8861d6398caaf079:0x7a735e53dfeac495,None,34.450636,-85.714130,[Barber shop],4.8,58,None,"[[Saturday, 10AM–5PM], [Sunday, Closed], [Mond...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 10AM,"[0x8861d640f8cf4415:0xc50ef1793d0439da, 0x8861...",https://www.google.com/maps/place//data=!4m2!3...
1932075,Mountain Travelers Hike and Ski Shop,"Mountain Travelers Hike and Ski Shop, 147 US-4...",0x89e024365cd93733:0x97f767d0409e00af,None,43.622089,-72.953674,"[Ski shop, Camping store, Car accessories store]",4.4,26,None,"[[Thursday, 10AM–5PM], [Friday, 10AM–5PM], [Sa...",None,Permanently closed,"[0x89e024ecbaaafc07:0xf913f5e468d2db08, 0x89e0...",https://www.google.com/maps/place//data=!4m2!3...
489691,Lovely Nails,"Lovely Nails, 636 Brandon Ave SW, Roanoke, VA ...",0x884d0da7e0e9de07:0xfaeb98dff1ad0e89,None,37.254535,-79.959470,[Nail salon],4.3,145,None,"[[Monday, 9:30AM–7:30PM], [Tuesday, 9:30AM–7:3...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 9:30AM Tue,"[0x884d0cff3546c2e1:0xf91b9ff679bd14aa, 0x884d...",https://www.google.com/maps/place//data=!4m2!3...


Información de los datos

In [24]:
data_type_check(df_metadata)


 Resumen del dataframe:

Dimensiones:  (220001, 15)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0               name      100.00     0.00            5    object
1            address       97.15     2.85         6275    object
2            gmap_id      100.00     0.00            0    object
3        description        7.16    92.84       204255    object
4           latitude      100.00     0.00            0   float64
5          longitude      100.00     0.00            0   float64
6           category       99.35     0.65         1428    object
7         avg_rating      100.00     0.00            0   float64
8     num_of_reviews      100.00     0.00            0     int64
9              price        7.68    92.32       203095    object
10             hours       72.82    27.18        59802    object
11              MISC       75.58    24.42        53728    object
12             state       74.25    25.75        56652    object
13  relative_results       89.49    1

In [32]:
filtro = df_metadata['description'].notnull()
df_metadata[filtro]

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
1216956,Trinity Bicycles,"Trinity Bicycles, 343 Throckmorton St, Fort Wo...",0x864e711216c9be97:0xec9c2f0f7fc64127,Full-service bike shop with sales & repairs of...,32.754782,-97.333209,"[Bicycle Shop, Bicycle rental service, Bicycle...",3.8,8,None,"[[Wednesday, 10AM–8PM], [Thursday, 10AM–8PM], ...",None,Permanently closed,"[0x864e6d8f0ad9b2db:0xb34e59726a9afc29, 0x864e...",https://www.google.com/maps/place//data=!4m2!3...
633645,Veranda Restaurant,"Veranda Restaurant, 444 St Charles Ave, New Or...",0x8620a674a5912e7b:0x3b5066bbdadc384c,Buffet spot in the Intercontinental Hotel offe...,29.950275,-90.069640,[Restaurant],1.0,2,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x8620a67c929a03f1:0xbc39f6aaa2af5c18, 0x8620...",https://www.google.com/maps/place//data=!4m2!3...
286874,Lids,"Lids, 9469 W Atlantic Blvd SP9149, Coral Sprin...",0x88d9051205135b51:0x323b4d368226f0e5,"Chain specializing in licensed team hats, logo...",26.241039,-80.249208,"[Hat shop, Embroidery service]",4.1,18,$$,"[[Wednesday, 11AM–8PM], [Thursday, 11AM–8PM], ...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 8PM,"[0x88d9031c25e1733b:0x6b83d9c09ec6f8be, 0x88d9...",https://www.google.com/maps/place//data=!4m2!3...
1215876,The UPS Store,"The UPS Store, 201 3rd Ave Bldg 38200, Fort Go...",0x88f9d7293111c42d:0x374bc8686df17345,"Store offering shipping, packaging, printing &...",33.407944,-82.135311,"[Shipping and mailing service, Business center...",2.2,18,None,"[[Wednesday, 9AM–6:30PM], [Thursday, 9AM–6:30P...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 9AM,"[0x88f9cdfe0d4e9e11:0x850c430c7a61c854, 0x88f9...",https://www.google.com/maps/place//data=!4m2!3...
493209,Ulta Beauty,"Ulta Beauty, 5228 S Lake Park Ave, Chicago, IL...",0x880e29718ddc36e1:0xbaf4e05bd1172021,Chain with a diverse selection of cosmetics & ...,41.799947,-87.587851,"[Cosmetics store, Beauty salon, Hairdresser]",4.0,178,None,"[[Monday, 10AM–6PM], [Tuesday, 10AM–8PM], [Wed...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 10AM Tue,"[0x880e2ca52e73a6b1:0xf1c4db851df1bdc8, 0x880e...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086659,Great Clips,"Great Clips, 1144 Shackelford Rd, Florissant, ...",0x87df3764bfe8f387:0x15fab92abe83871e,Casual salon offering haircuts for adults & ki...,38.812564,-90.347582,"[Hair salon, Beauty salon]",4.2,64,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x87df37a357af6ac7:0x544294f68581f351, 0x87df...",https://www.google.com/maps/place//data=!4m2!3...
2091817,Subway Restaurants,"Subway Restaurants, 4836 Paramount Blvd, Lakew...",0x80dd3301c4a62307:0x294b1f7d5d6f9aff,Casual counter-serve chain for build-your-own ...,33.846705,-118.159031,[Hoagie restaurant],3.6,18,$,"[[Monday, 7AM–10PM], [Tuesday, 7AM–10PM], [Wed...","{'Accessibility': None, 'Activities': None, 'A...",Permanently closed,"[0x80c2d2c43d9b37f7:0x4f3469991b7ca958, 0x80dd...",https://www.google.com/maps/place//data=!4m2!3...
845471,Spettro,"Spettro, 3355 Lakeshore Ave, Oakland, CA 94610",0x808f8741facdeff9:0x71019ed1fc485171,Italian spot with no corkage fee & innovative ...,37.811351,-122.243358,"[Italian restaurant, Restaurant]",4.0,27,$$,"[[Tuesday, 5–9:30PM], [Wednesday, 5–9:30PM], [...","{'Accessibility': None, 'Activities': None, 'A...",Permanently closed,"[0x808f876a0768f0d5:0x177c1d11da690133, 0x808f...",https://www.google.com/maps/place//data=!4m2!3...
1673766,Starbucks,"Starbucks, 5963 Corson Ave S building a unit 1...",0x549041eb316f1f1b:0x5a981c5300ff3288,Seattle-based coffeehouse chain known for its ...,47.548112,-122.322196,"[Coffee shop, Breakfast restaurant, Cafe, Coff...",4.1,134,$$,"[[Wednesday, 4AM–6PM], [Thursday, 4AM–6PM], [F...",{'Accessibility': ['Wheelchair accessible entr...,Permanen